Library Import

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

from sklearn.decomposition import PCA

from sklearn. preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score

import pickle

Data loading

In [ ]:
dataframe = pd.read_csv('/content/collegePlace.csv.csv')

In [ ]:
# shape of data
dataframe.shape

(2966, 8)

In [ ]:
# first 5 rows of the dataset
dataframe.head()

,Age,Gender,Stream,Internships,CGPA,Hostel,HistoryOfBacklogs,PlacedOrNot
0,22,Male,Electronics And Communication,1,8,1,1,1
1,21,Female,Computer Science,0,7,1,1,1
2,22,Female,Information Technology,1,6,0,0,1
3,21,Male,Information Technology,0,8,0,1,1
4,22,Male,Mechanical,0,8,1,0,1


In [ ]:
# Showing 4 rows of the dataset at random
dataframe.sample(4)

,Age,Gender,Stream,Internships,CGPA,Hostel,HistoryOfBacklogs,PlacedOrNot
2716,21,Male,Computer Science,0,8,0,0,1
2242,19,Female,Electronics And Communication,1,8,0,1,1
2564,21,Male,Mechanical,0,6,1,0,0
1666,24,Male,Computer Science,1,8,0,0,1


In [ ]:
# Getting to know the data type of columns that are in the dataset
dataframe.dtypes

Age                   int64
Gender               object
Stream               object
Internships           int64
CGPA                  int64
Hostel                int64
HistoryOfBacklogs     int64
PlacedOrNot           int64
dtype: object

In [ ]:
# Getting to know the detailed information of the columns
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2966 entries, 0 to 2965
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Age                2966 non-null   int64 
 1   Gender             2966 non-null   object
 2   Stream             2966 non-null   object
 3   Internships        2966 non-null   int64 
 4   CGPA               2966 non-null   int64 
 5   Hostel             2966 non-null   int64 
 6   HistoryOfBacklogs  2966 non-null   int64 
 7   PlacedOrNot        2966 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 185.5+ KB


In [ ]:
# Statistical Descriptions of the numerical values in the dataset
dataframe.describe()

,Age,Internships,CGPA,Hostel,HistoryOfBacklogs,PlacedOrNot
count,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000
mean,21.485840,0.703641,7.073837,0.269049,0.192178,0.552596
std,1.324933,0.740197,0.967748,0.443540,0.394079,0.497310
min,19.000000,0.000000,5.000000,0.000000,0.000000,0.000000
25%,21.000000,0.000000,6.000000,0.000000,0.000000,0.000000
50%,21.000000,1.000000,7.000000,0.000000,0.000000,1.000000
75%,22.000000,1.000000,8.000000,1.000000,0.000000,1.000000
max,30.000000,3.000000,9.000000,1.000000,1.000000,1.000000


In [ ]:
# Getting to know the correlation between the target column and other features.
dataframe.corr()['PlacedOrNot']

<ipython-input-48-59b32fc1d4ba>:2: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



Age                  0.046943
Internships          0.179334
CGPA                 0.588648
Hostel              -0.038182
HistoryOfBacklogs   -0.022337
PlacedOrNot          1.000000
Name: PlacedOrNot, dtype: float64

In [ ]:
# missing values
dataframe.isnull().sum()


Age                  0
Gender               0
Stream               0
Internships          0
CGPA                 0
Hostel               0
HistoryOfBacklogs    0
PlacedOrNot          0
dtype: int64

In [ ]:
# duplicate rows
print(dataframe.duplicated().sum())

#drop duplicates
dataframe.drop_duplicates(inplace=True)

1829


In [ ]:
# Check if the duplicate rows are removed
print(dataframe.duplicated().sum())

0


EDA

In [ ]:
# Plotting  the graph so that we can visualize the output with respect to major features
figure = px.scatter(dataframe, x="CGPA", y="Internships", color="PlacedOrNot",
                 hover_data=['CGPA'])
figure.show()

In [ ]:
# Plotting Histogram for the count of place and not placed
px.histogram(dataframe, x='PlacedOrNot', color='PlacedOrNot', barmode='group')

In [ ]:
# Pie Chart: Percentage pie chart of Placed or Not Placed
figure = px.pie(dataframe, values=dataframe['PlacedOrNot'].value_counts().values, names=dataframe['PlacedOrNot'].value_counts().index, title='Placed Vs Not Placed')
figure.show()

In [ ]:
# Printing the Age of the youngest and Eldest student who is placed
print("Max Age of Placed Person: ",dataframe[(dataframe['Age'] == dataframe['Age'].max()) & (dataframe['PlacedOrNot']==1)]['Age'].values[0])
print("Min Age of Placed Person: ",dataframe[(dataframe['Age'] == dataframe['Age'].min()) & (dataframe['PlacedOrNot']==1)]['Age'].values[0])

Max Age of Placed Person:  30
Min Age of Placed Person:  19


In [ ]:
# Printing the Maximum and the Minimum number of internships done by the student who is placed.
# We will also print the Maximum and Minimum number of students who did the max internship and the minimum number of internships.
print("Max Internships Done by the Placed Student: ",dataframe[(dataframe['Internships'] == dataframe['Internships'].max()) & (dataframe['PlacedOrNot']==1)]['Internships'].values[0])
print("No of students who did max Internships and are placed: ",dataframe[(dataframe['Internships'] == dataframe['Internships'].max()) & (dataframe['PlacedOrNot']==1)]['Internships'].value_counts().values[0])

print("Min Internships Done by the Placed Person: ",dataframe[(dataframe['Internships'] == dataframe['Internships'].min()) & (dataframe['PlacedOrNot']==1)]['Internships'].values[0])
print("No of students who did min Internships and are placed: ",dataframe[(dataframe['Internships'] == dataframe['Internships'].min()) & (dataframe['PlacedOrNot']==1)]['Internships'].value_counts().values[0])

Max Internships Done by the Placed Student:  3
No of students who did max Internships and are placed:  35
Min Internships Done by the Placed Person:  0
No of students who did min Internships and are placed:  232


In [ ]:
# Printing the Maximum and Minimum number of CGPA obtained by the student who is placed.
# We will also print the Maximum and the Minimum number of students who got the max CGPA and minimum CGPA who are placed.

print("Max CGPA of Placed Student: ",dataframe[(dataframe['CGPA'] == dataframe['CGPA'].max()) & (dataframe['PlacedOrNot']==1)]['CGPA'].values[0])
print("No of students has max CGPA and are placed: ",dataframe[(dataframe['CGPA'] == dataframe['CGPA'].max()) & (dataframe['PlacedOrNot']==1)]['CGPA'].value_counts().values[0])

print("Min CGPA of Placed Person: ",dataframe[(dataframe['CGPA'] == dataframe['CGPA'].min()) & (dataframe['PlacedOrNot']==1)]['CGPA'].values[0])
print("No of students has min CGPA and are placed: ",dataframe[(dataframe['CGPA'] == dataframe['CGPA'].min()) & (dataframe['PlacedOrNot']==1)]['CGPA'].value_counts().values[0])

Max CGPA of Placed Student:  9
No of students has max CGPA and are placed:  99
Min CGPA of Placed Person:  5
No of students has min CGPA and are placed:  5


EDA

In [ ]:
figure = px.box(dataframe, y='CGPA')
figure.show()

In [ ]:
figure = px.box(dataframe, y='Age')
figure.show()

In [ ]:
figure = px.box(dataframe, y=['Internships','CGPA', 'Age'])
figure.show()

In [ ]:
# Converting Gender column
dataframe['Gender'] = dataframe['Gender'].map({'Male': 1, 'Female': 0})

In [ ]:
dataframe['Stream'].unique()

array(['Electronics And Communication', 'Computer Science',
       'Information Technology', 'Mechanical', 'Electrical', 'Civil'],
      dtype=object)

In [ ]:
# Converting Stream column
dataframe['Stream'] = dataframe['Stream'].map({'Electronics And Communication': 1,
                                 'Computer Science': 2,
                                'Information Technology': 3,
                                'Mechanical':4,
                                'Electrical':5,
                                'Civil':6})

In [ ]:
dataframe.sample(5)

,Age,Gender,Stream,Internships,CGPA,Hostel,HistoryOfBacklogs,PlacedOrNot
2090,19,1,1,0,6,0,0,0
359,22,1,4,0,7,1,0,1
2944,23,1,3,1,7,0,1,0
1780,22,0,1,1,7,1,0,0
464,22,1,3,1,6,1,0,0


In [ ]:
# creating new data frame
X = dataframe.iloc[:,0:7]
y = dataframe.iloc[:,-1]
X

,Age,Gender,Stream,Internships,CGPA,Hostel,HistoryOfBacklogs
0,22,1,1,1,8,1,1
1,21,0,2,0,7,1,1
2,22,0,3,1,6,0,0
3,21,1,3,0,8,0,1
4,22,1,4,0,8,1,0
...,...,...,...,...,...,...,...
2946,23,1,3,1,7,1,1
2952,23,1,4,0,8,1,0
2954,23,0,2,1,8,0,1
2958,23,1,2,0,6,0,1


In [ ]:
# Getting the shape of the X and Y
print(X.shape)
print(y.shape)

(1137, 7)
(1137,)


In [ ]:
# Splitting the dataset into training and testing datasets.
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33)

In [ ]:
# Getting the Shape of all the training and testing dataset
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(761, 7)
(376, 7)
(761,)
(376,)


In [ ]:
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

# Without Scaling
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(classifier, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling
classifier.fit(X_train_scale,y_train)
y_pred = classifier.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(classifier, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())

#scaling has not much effect

Without Scaling and CV:  0.726063829787234


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



Without Scaling and With CV:  0.7109022556390977
With Scaling and Without CV:  0.726063829787234
With Scaling and With CV:  0.712218045112782


Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(random_state=0)

#without scaling
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(classifier, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling
classifier.fit(X_train_scale,y_train)
y_pred = classifier.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(classifier, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())

Without Scaling and CV:  0.7446808510638298
Without Scaling and With CV:  0.7293233082706767
With Scaling and Without CV:  0.7446808510638298
With Scaling and With CV:  0.7293233082706767


Random Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(max_depth=10, random_state=0)

classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(classifier, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling
classifier.fit(X_train_scale,y_train)
y_pred = classifier.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(classifier, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())

Without Scaling and CV:  0.7845744680851063
Without Scaling and With CV:  0.7844839371155161
With Scaling and Without CV:  0.7898936170212766
With Scaling and With CV:  0.7858168147641832


Support vector machine

In [ ]:
from sklearn.svm import SVC

svc = SVC()
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
classifier = GridSearchCV(svc, parameters)

classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(classifier, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling
classifier.fit(X_train_scale,y_train)
y_pred = classifier.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(classifier, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())

Without Scaling and CV:  0.7473404255319149
Without Scaling and With CV:  0.724025974025974
With Scaling and Without CV:  0.8138297872340425
With Scaling and With CV:  0.7621667805878333


KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=3)

classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(classifier, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())


# With Scaling
classifier.fit(X_train_scale,y_train)
y_pred = classifier.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(classifier, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())

Without Scaling and CV:  0.7340425531914894
Without Scaling and With CV:  0.7254272043745728
With Scaling and Without CV:  0.726063829787234
With Scaling and With CV:  0.6873034859876965


Model Selection

In [ ]:
classifier = RandomForestClassifier(max_depth=10, random_state=0)
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)
print("Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(classifier, X_train, y_train, cv=10)
print("With CV: ",scores.mean())
print("Precision Score: ", precision_score(y_test, y_pred))
print("Recall Score: ", recall_score(y_test, y_pred))
print("F1 Score: ", f1_score(y_test, y_pred))

Without CV:  0.7845744680851063
With CV:  0.7844839371155161
Precision Score:  0.8509615384615384
Recall Score:  0.7797356828193832
F1 Score:  0.8137931034482758


Model tuning

In [ ]:
param_grid = {
    'bootstrap': [False,True],
    'max_depth': [5,8,10, 20],
    'max_features': [3, 4, 5, None],
    'min_samples_split': [2, 10, 12],
    'n_estimators': [100, 200, 300]
}

rfclassifier = RandomForestClassifier()

classifier = GridSearchCV(estimator=rfclassifier, param_grid=param_grid, cv=5, n_jobs=-1, verbose=1)

classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred))
print(classifier.best_params_)
print(classifier.best_estimator_)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits
Accuracy:  0.8218085106382979
{'bootstrap': True, 'max_depth': 8, 'max_features': 5, 'min_samples_split': 10, 'n_estimators': 200}
RandomForestClassifier(max_depth=8, max_features=5, min_samples_split=10,
                       n_estimators=200)


Precision Score

In [ ]:
classifier = RandomForestClassifier(bootstrap=False, max_depth=5,max_features=None,
                             min_samples_split=2,
                             n_estimators=100, random_state=0)
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)
print("Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(classifier, X_train, y_train, cv=10)
print("With CV: ",scores.mean())
print("Precision Score: ", precision_score(y_test, y_pred))
print("Recall Score: ", recall_score(y_test, y_pred))
print("F1 Score: ", f1_score(y_test, y_pred))

Without CV:  0.8351063829787234
With CV:  0.8226247436773753
Precision Score:  0.9508196721311475
Recall Score:  0.7665198237885462
F1 Score:  0.848780487804878
